In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.maskers import NiftiLabelsMasker
from nilearn import datasets, image

# === Define Subject IDs ===
# subject_ids = [
#     "sub-0001", "sub-0002", "sub-0003", "sub-0004", "sub-0005",
#     "sub-0006", "sub-0007", "sub-0008", "sub-0009", "sub-0011"
# ]
# subject_ids = [
#     "sub-0001", "sub-0002"
# ]
subject_ids = [
    "sub-0001"
]

In [2]:
import nibabel as nib
import matplotlib.pyplot as plt

# === Load NIfTI Image ===
img_task = nib.load("sub-0001_task-workingmemory_acq-seq_bold.nii.gz")
data_task = img_task.get_fdata()  # Shape: (X, Y, Z, T)
header_task = img_task.header
print("Header of fMRI task data:",header_task.get_zooms())
print("Shape of fMRI taskdata:", data_task.shape)

Header of fMRI task data: (3.0, 3.0, 3.3, 2.0)
Shape of fMRI taskdata: (80, 80, 36, 160)


In [3]:
img_rest = nib.load("sub-0001_task-restingstate_acq-seq_bold.nii.gz")
data_rest = img_rest.get_fdata()  # Shape: (X, Y, Z, T)
header_rest = img_rest.header
print("Header of fMRI rest data:",header_rest.get_zooms())
print("Shape of fMRI rest data:", data_rest.shape)

Header of fMRI rest data: (3.0, 3.0, 3.3, 2.0)
Shape of fMRI rest data: (80, 80, 36, 240)


In [4]:
# === Construct File Paths ===
fmri_rest_files = [f"{sid}_task-restingstate_acq-seq_bold.nii.gz" for sid in subject_ids]
fmri_task_files = [f"{sid}_task-workingmemory_acq-seq_bold.nii.gz" for sid in subject_ids]
tsv_files = [f"{sid}.tsv" for sid in subject_ids]


In [5]:
# === Load Atlas and Define Masker ===
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100)
masker = NiftiLabelsMasker(atlas.maps, standardize=True)


[get_dataset_dir] Dataset found in C:\Users\elena\nilearn_data\schaefer_2018


In [8]:
import numpy as np
import pandas as pd

def build_trial_wise_matrix(time_series_task, time_series_rest, tsv_path, TR=2.0, fixed_len=3):
    """
    Επιστρέφει αντίστοιχα blocks task και rest για κάθε trial, με βάση τα onset του .tsv.

    Parameters:
    - time_series_task: NumPy array (T_task, R)
    - time_series_rest: NumPy array (T_rest, R)
    - tsv_path: string, path προς .tsv αρχείο
    - TR: χρονική διάρκεια TR σε δευτερόλεπτα
    - fixed_len: αριθμός TRs ανά trial block (default: 3)

    Returns:
    - task_matrix: NumPy array (n_trials, fixed_len, n_rois)
    - rest_matrix: NumPy array (n_trials, fixed_len, n_rois)
    """
    df = pd.read_csv(tsv_path, sep="\t")
    n_timepoints_task, n_rois = time_series_task.shape
    n_timepoints_rest = time_series_rest.shape[0]

    task_matrix = []
    rest_matrix = []

    for _, row in df.iterrows():
        onset = row["onset"]
        start_tr = int(np.floor(onset / TR))
        end_tr = start_tr + fixed_len

        # Skip trial αν δεν χωράει ούτε σε task ούτε σε rest
        if end_tr > n_timepoints_task or end_tr > n_timepoints_rest:
            continue

        block_task = time_series_task[start_tr:end_tr]
        block_rest = time_series_rest[start_tr:end_tr]

        task_matrix.append(block_task)
        rest_matrix.append(block_rest)

    return np.array(task_matrix), np.array(rest_matrix)


In [9]:
subject_task_data = {}
subject_rest_data = {}

for sid, rest_file, task_file, tsv_file in zip(subject_ids, fmri_rest_files, fmri_task_files, tsv_files):
    print(f"Processing {sid}")

    img_task = nib.load(task_file)
    img_rest = nib.load(rest_file)

    ts_task = masker.fit_transform(img_task)
    ts_rest = masker.transform(img_rest)

    task_matrix, rest_matrix = build_trial_wise_matrix(ts_task, ts_rest, tsv_file)

    subject_task_data[sid] = task_matrix
    subject_rest_data[sid] = rest_matrix

Processing sub-0001


In [10]:
for sid,sid in zip(subject_task_data,subject_rest_data):
    print(f"Subject {sid} task data shape",subject_task_data[sid].shape)
    print(f"Subject {sid} rest data shape",subject_rest_data[sid].shape)

Subject sub-0001 task data shape (40, 3, 100)
Subject sub-0001 rest data shape (40, 3, 100)


In [11]:
import numpy as np

def compute_roi_correlation_matrix(data):
    """
    Υπολογίζει Pearson correlation matrix μεταξύ ROIs,
    flattening όλα τα TRs από όλα τα trials.

    Parameters:
    - data: NumPy array (n_trials, 3, n_rois)

    Returns:
    - corr_matrix: NumPy array (n_rois, n_rois)
    """
    n_trials, n_trs, n_rois = data.shape
    flat_data = data.reshape(-1, n_rois)  # shape: (n_trials * 3, n_rois)

    corr_matrix = np.corrcoef(flat_data.T)  # (n_rois, n_rois)
    return corr_matrix


In [12]:
task_corr_dict = {}
rest_corr_dict = {}

for sid in subject_ids:
    task_corr = compute_roi_correlation_matrix(subject_task_data[sid])
    rest_corr = compute_roi_correlation_matrix(subject_rest_data[sid])

    task_corr_dict[sid]=task_corr
    rest_corr_dict[sid]=rest_corr


for sid,sid in zip(task_corr_dict,rest_corr_dict):
    print(f"Subject {sid} task data correlation",task_corr_dict[sid].shape)
    print(f"Subject {sid} rest data correlation",task_corr_dict[sid].shape)


Subject sub-0001 task data correlation (100, 100)
Subject sub-0001 rest data correlation (100, 100)


In [13]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

def compute_partial_pearson_matrix(data, num_components=3):
    """
    Υπολογίζει πίνακα Partial Pearson Correlation μεταξύ ROIs,
    αφαιρώντας κοινή πληροφορία μέσω PCA (σε flatten δεδομένα από trials).

    Parameters:
    - data: NumPy array (n_trials, n_TRs, n_rois)
    - num_components: Αριθμός PCA components που θα αφαιρεθούν ως confounds

    Returns:
    - partial_corr_matrix: NumPy array (n_rois, n_rois) συμμετρικός
    """
    n_trials, n_trs, n_rois = data.shape
    flat_data = data.reshape(-1, n_rois)

    partial_corr_matrix = np.eye(n_rois)

    for i in range(n_rois):
        for j in range(i + 1, n_rois):
            X = flat_data[:, i].reshape(-1, 1)
            Y = flat_data[:, j].reshape(-1, 1)

            other_indices = [k for k in range(n_rois) if k != i and k != j]
            Z = flat_data[:, other_indices]

            num_components
            Z_pca = PCA(n_components=num_components).fit_transform(Z)

            def regress_out(A, Z_pca):
                reg = LinearRegression().fit(Z_pca, A)
                return A - reg.predict(Z_pca)

            X_resid = regress_out(X, Z_pca).flatten()
            Y_resid = regress_out(Y, Z_pca).flatten()

            corr = np.corrcoef(X_resid, Y_resid)[0, 1]
            partial_corr_matrix[i, j] = corr
            partial_corr_matrix[j, i] = corr

    return partial_corr_matrix

In [14]:
task_part_corr_dict = {}
rest_part_corr_dict = {}

for sid in subject_ids:
    task_part_corr = compute_partial_pearson_matrix(subject_task_data[sid])
    rest_part_corr = compute_partial_pearson_matrix(subject_rest_data[sid])

    task_part_corr_dict[sid]=task_part_corr
    rest_part_corr_dict[sid]=rest_part_corr
    print(f"Subject {sid} task data partial correlation",task_part_corr_dict[sid].shape)
    print(f"Subject {sid} rest data partial correlation",rest_part_corr_dict[sid].shape)


Subject sub-0001 task data partial correlation (100, 100)
Subject sub-0001 rest data partial correlation (100, 100)


In [30]:
import numpy as np
from sklearn.cross_decomposition import CCA

def compute_cca_matrix_across_trials(data, n_components=1):
    """
    Υπολογίζει Canonical Correlation μεταξύ κάθε ζεύγους ROIs,
    χωρίς flatten — χρησιμοποιώντας τα 3 TRs ανά trial ως μεταβλητές.

    Parameters:
    - data: NumPy array (n_trials, 3, n_rois)
    - n_components: αριθμός CCA components (συνήθως 1)

    Returns:
    - cca_matrix: NumPy array (n_rois, n_rois), συμμετρικός
    """
    n_trials, n_trs, n_rois = data.shape
    cca_matrix = np.eye(n_rois)

    for i in range(n_rois):
        for j in range(i + 1, n_rois):
            X = data[:, :, i]  # shape: (n_trials, 3)
            Y = data[:, :, j]  # shape: (n_trials, 3)

            cca = CCA(n_components=n_components)
            X_c, Y_c = cca.fit_transform(X, Y)

            # Υπολογίζουμε την Pearson correlation του 1ου canonical pair
            corr = np.corrcoef(X_c[:, 0], Y_c[:, 0])[0, 1]
            cca_matrix[i, j] = corr
            cca_matrix[j, i] = corr

    return cca_matrix


In [31]:
import numpy as np
from scipy.linalg import eig,pinv

def compute_cca_matrix_across_trials_linear(data, n_components=1):
    """
    Υπολογίζει Canonical Correlation μεταξύ κάθε ζεύγους ROIs,
    με χρήση αλγεβρας (χωρίς sklearn), across trials.

    Parameters:
    - data: NumPy array (n_trials, 3, n_rois)
    - n_components: αριθμός CCA components (συνήθως 1)

    Returns:
    - cca_matrix: NumPy array (n_rois, n_rois), συμμετρικός
    """
    n_trials, n_trs, n_rois = data.shape
    cca_matrix = np.eye(n_rois)

    def center(A):
        return A - A.mean(axis=0, keepdims=True)

    def cov(A, B):
        A = center(A)
        B = center(B)
        return A.T @ B / (A.shape[0] - 1)

    for i in range(n_rois):
        for j in range(i + 1, n_rois):
            # Each ROI is (n_trials, 3)
            X = data[:, :, i]
            Y = data[:, :, j]

            # Centered covariance matrices
            Sxx = cov(X, X)
            Syy = cov(Y, Y)
            Sxy = cov(X, Y)
            Syx = Sxy.T

            # Form generalized eigenvalue problem
            A = np.block([
                [np.zeros_like(Sxy), Sxy],
                [Syx, np.zeros_like(Syx)]
            ])
            B = np.block([
                [Sxx, np.zeros_like(Sxy)],
                [np.zeros_like(Syx), Syy]
            ])

            # Solve generalized eigenproblem
            eigvals, _ = eig(A, B)
            eigvals = np.real(eigvals)
            canonical_corrs = np.sort(np.abs(eigvals))[::-1]

            rho = canonical_corrs[0] if canonical_corrs.size > 0 else 0.0
            cca_matrix[i, j] = rho
            cca_matrix[j, i] = rho

    return cca_matrix

In [ ]:
for sid in subject_ids:
    corr1 = compute_cca_matrix_across_trials_linear(subject_task_data[sid])
    corr2 = compute_cca_matrix_across_trials(subject_task_data[sid])
    # print("Custom:", corr1, "Sklearn:", corr2)
print("Max diff:", np.abs(corr1 - corr2).max())

Custom: [[1.         0.51553911 0.73055565 ... 0.73033738 0.77613211 0.66972917]
 [0.51553911 1.         0.83662915 ... 0.69353741 0.37447917 0.48290861]
 [0.73055565 0.83662915 1.         ... 0.64702463 0.75728619 0.61242187]
 ...
 [0.73033738 0.69353741 0.64702463 ... 1.         0.61375815 0.6584022 ]
 [0.77613211 0.37447917 0.75728619 ... 0.61375815 1.         0.82249635]
 [0.66972917 0.48290861 0.61242187 ... 0.6584022  0.82249635 1.        ]] Sklearn: [[1.         0.51553896 0.73055538 ... 0.73033723 0.77613124 0.66972911]
 [0.51553896 1.         0.83662899 ... 0.69353735 0.37447888 0.48290842]
 [0.73055538 0.83662899 1.         ... 0.64702457 0.75728608 0.61242165]
 ...
 [0.73033723 0.69353735 0.64702457 ... 1.         0.61375786 0.65840215]
 [0.77613124 0.37447888 0.75728608 ... 0.61375786 1.         0.82249581]
 [0.66972911 0.48290842 0.61242165 ... 0.65840215 0.82249581 1.        ]]
Max diff: 0.0032532972263871196


In [32]:
task_cca_dict = {}
rest_cca_dict = {}

for sid in subject_ids:
    task_cca = compute_cca_matrix_across_trials(subject_task_data[sid])
    rest_cca = compute_cca_matrix_across_trials(subject_rest_data[sid])

    task_cca_dict[sid]=task_cca
    rest_cca_dict[sid]=rest_cca
    print(f"Subject {sid} task data cca",task_cca_dict[sid].shape)
    print(f"Subject {sid} rest data cca",task_cca_dict[sid].shape)

Subject sub-0001 task data cca (100, 100)
Subject sub-0001 rest data cca (100, 100)


In [33]:
from scipy.linalg import eig

def compute_pcca_per_trial_with_topkZ(trial_data, i, j, cca_matrix, k=2):
    """
    Computes PCCA between ROI i and j in a single trial,
    using top-k confounding ROIs selected from cca_matrix.

    Parameters:
    - trial_data: shape (3, n_rois) for one trial
    - i, j: indices of X and Y ROIs
    - cca_matrix: shape (n_rois, n_rois)
    - k: number of ROIs to use in Z

    Returns:
    - pcca_rho: partial canonical correlation (scalar)
    """
    X = trial_data[:, i].reshape(-1, 1)
    Y = trial_data[:, j].reshape(-1, 1)

    # Select top-k confounding ROIs based on CCA matrix
    candidate_indices = [z for z in range(trial_data.shape[1]) if z != i and z != j]
    relevance_scores = [max(cca_matrix[i, z], cca_matrix[j, z]) for z in candidate_indices]
    top_k_indices = [candidate_indices[z] for z in np.argsort(relevance_scores)[-k:]]

    Z = trial_data[:, top_k_indices]  # shape (3, k)

    # Covariance helpers
    def center(A): return A - A.mean(axis=0, keepdims=True)
    def cov(A, B): return center(A).T @ center(B) / (A.shape[0] - 1)

    # Covariances
    Sxz = cov(X, Z)
    Syz = cov(Y, Z)
    Szz = cov(Z, Z)
    Sxy = cov(X, Y)
    Sxx = cov(X, X)
    Syy = cov(Y, Y)

    # Partial covariances
    inv_Szz = np.linalg.pinv(Szz)
    Sxy_z = Sxy - Sxz @ inv_Szz @ Syz.T
    Syx_z = Sxy_z.T
    Sxx_z = Sxx - Sxz @ inv_Szz @ Sxz.T
    Syy_z = Syy - Syz @ inv_Szz @ Syz.T

    # Generalized eigenvalue problem
    A = np.block([
        [np.zeros_like(Sxy_z), Sxy_z],
        [Syx_z, np.zeros_like(Syx_z)]
    ])
    B = np.block([
        [Sxx_z, np.zeros_like(Sxy_z)],
        [np.zeros_like(Syx_z), Syy_z]
    ])

    eigvals, _ = eig(A, B)
    eigvals = np.real(eigvals)
    canonical_corrs = np.sort(np.abs(eigvals))[::-1]

    return canonical_corrs[0] if canonical_corrs.size > 0 else 0.0

In [34]:
def compute_pcca_tensor(data, cca_matrix, k=2):
    """
    Computes the full PCCA tensor:
    shape = (n_trials, n_rois, n_rois)

    Parameters:
    - data: shape (n_trials, 3, n_rois)
    - cca_matrix: shape (n_rois, n_rois), from compute_cca_matrix_across_trials
    - k: number of top confounding ROIs to use in Z

    Returns:
    - pcca_tensor: (n_trials, n_rois, n_rois)
    """
    n_trials, _, n_rois = data.shape
    pcca_tensor = np.zeros((n_trials, n_rois, n_rois))

    for t in range(n_trials):
        trial = data[t]  # shape (3, n_rois)
        for i in range(n_rois):
            for j in range(i + 1, n_rois):
                pcca_val = compute_pcca_per_trial_with_topkZ(trial, i, j, cca_matrix, k)
                pcca_tensor[t, i, j] = pcca_val
                pcca_tensor[t, j, i] = pcca_val  # symmetric

    return pcca_tensor

In [ ]:
task_pcca_dict = {}
rest_pcca_dict = {}

for sid in subject_ids:
    task_pcca = compute_pcca_tensor(subject_task_data[sid],task_cca_dict[sid])
    rest_pcca = compute_pcca_tensor(subject_rest_data[sid],rest_cca_dict[sid])

    task_pcca_dict[sid]=task_pcca
    rest_pcca_dict[sid]=rest_pcca
    print(f"Subject {sid} task data pcca",task_pcca_dict[sid].shape)
    print(f"Subject {sid} rest data pcca",rest_pcca_dict[sid].shape)